In [ ]:
from datasets import load_dataset
import json

ds = load_dataset("BAAI/InfinityMATH", streaming=True)
train = ds["train"]

with open("data/raw/train.jsonl", "w", encoding="utf-8") as f:
    for row in train:
        filtered = {
            "original_problem": row.get("original_problem"),
            "variables": row.get("variables")
        }
        f.write(json.dumps(filtered, ensure_ascii=False) + "\n")

In [9]:
from datasets import load_dataset
import json

def safe(x):
    # variables đôi khi có thể là kiểu lạ -> ép về dạng JSON-safe
    if x is None:
        return None
    if isinstance(x, (str, int, float, bool, list, dict)):
        return x
    return str(x)

ds = load_dataset("BAAI/InfinityMATH", streaming=True)
train = ds["train"]

with open("data/raw/train.jsonl", "w", encoding="utf-8") as f:
    for row in train:
        out = {
            "original_problem": safe(row.get("original_problem")),
            "variables": safe(row.get("variables")),
        }
        f.write(json.dumps(out, ensure_ascii=False) + "\n")


Failed to load JSON from file 'hf://datasets/BAAI/InfinityMATH@305f461470cde82537934f01ec0d09f43a0dce98/infinitymath.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column(/ori_answer) changed from number to string in row 16


ArrowInvalid: JSON parse error: Column(/ori_answer) changed from number to string in row 16

In [1]:
import requests
import json

def export_infinitymath_two_fields(
    output_path="data/raw/train.jsonl"
):
    url = "https://huggingface.co/datasets/BAAI/InfinityMATH/resolve/main/infinitymath.jsonl"
    count = 0

    with requests.get(url, stream=True) as r:
        r.raise_for_status()

        with open(output_path, "w", encoding="utf-8") as f:
            for line in r.iter_lines():
                if not line:
                    continue

                try:
                    obj = json.loads(line)

                    filtered = {
                        "original_problem": obj.get("original_problem"),
                        "variables": obj.get("variables"),
                    }

                    f.write(json.dumps(filtered, ensure_ascii=False) + "\n")
                    count += 1

                except json.JSONDecodeError:
                    continue  # bỏ qua dòng lỗi

    print(f"✅ Done! Saved {count} samples to {output_path}")


# Chạy
export_infinitymath_two_fields()


✅ Done! Saved 101380 samples to data/raw/train.jsonl
